In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../datasets/Social_Network_Ads.csv')
df.head()

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0


In [14]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

X = df.iloc[:, 2:-1].values
y = df.iloc[:, -1].values

x_scaler = StandardScaler()
X = x_scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [18]:
classifier = SVC(kernel='rbf')
classifier.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [35]:
# Applying Grid Search to find the best model and the best hyperparameters
from sklearn.model_selection import GridSearchCV

parameters = [
    {
        'C': [1, 10, 100, 1000], 
        'kernel': ['linear']
    },
    {
        'C': [1, 10, 100, 1000], 
        'kernel': ['rbf'], 
        'gamma': [1, 0.975, 0.96, 0.958, 0.955, 0.953, 0.95, 0.945, 0.94, 0.925, 
                  0.9, 0.85, 0.8, 0.75, 0.7, 0.65, 0.6, 
                  0.5, 0.4, 0.3, 0.2, 0.1, 0.01, 0.001, 0.0001]
    }
]

grid_search = GridSearchCV(estimator=classifier, 
                           param_grid=parameters, 
                           scoring='accuracy', 
                           cv=10, 
                           n_jobs=-1)

grid_search = grid_search.fit(X_train, y_train)

best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_

print(f'best accuracy: {best_accuracy}')
print(best_parameters)

best accuracy: 0.909375
{'C': 1, 'gamma': 0.955, 'kernel': 'rbf'}


In [37]:
classifier = SVC(kernel='rbf', C=1, gamma=0.955)
classifier.fit(X_train, y_train)
accuracy = classifier.score(X_test, y_test)
cm = confusion_matrix(y_test, classifier.predict(X_test))
k_fold_accuracies = cross_val_score(estimator=classifier, X=X_train, y=y_train, cv=10)

print(f'accuracy: {accuracy}')
print(cm)
print()
print(f'10-fold mean accuracy: {k_fold_accuracies.mean()}')
print(f'10-fold st. deviation: {k_fold_accuracies.std()}')

accuracy: 0.95
[[55  3]
 [ 1 21]]

10-fold mean accuracy: 0.909375
10-fold st. deviation: 0.045177600921253
